In [ ]:
!pip install spacy==2.1.0
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.1.0/en_core_web_lg-2.1.0.tar.gz
!pip install neuralcoref
!pip install scikit-multilearn

#Note: Once en_core_web_lg-2.1.0 is installed, go to Runtime > Restart Runtime in order for your notebook to be able to find the model.

     |████████████████████████████████| 27.7MB 131kB/s 
     |████████████████████████████████| 92kB 9.5MB/s 
     |████████████████████████████████| 3.2MB 43.9MB/s 
     |████████████████████████████████| 2.1MB 35.8MB/s 
ERROR: en-core-web-sm 2.2.5 has requirement spacy>=2.2.2, but you'll have spacy 2.1.0 which is incompatible.
  Found existing installation: plac 1.1.3
    Uninstalling plac-1.1.3:
      Successfully uninstalled plac-1.1.3
  Found existing installation: preshed 3.0.2
    Uninstalling preshed-3.0.2:
      Successfully uninstalled preshed-3.0.2
  Found existing installation: blis 0.4.1
    Uninstalling blis-0.4.1:
      Successfully uninstalled blis-0.4.1
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 826.9MB 19kB/s 
  Created wheel for en-core-web-l

In [ ]:
import spacy
import neuralcoref
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from skmultilearn.problem_transform import LabelPowerset
import numpy as np
import pickle
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
import re
import string
import time
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.parse.stanford import StanfordDependencyParser
from nltk import Tree
from nltk import word_tokenize
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

100%|██████████| 40155833/40155833 [00:01<00:00, 25379300.86B/s]


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
def replace_pronouns(text):
    doc = nlp(text)
    return doc._.coref_resolved

def split_sentence(text):
    doc = sent_tokenize(text)
    for idx, item in enumerate(doc):
      if len(item) > 500:
        arrayTexts = item.split(".")
        if len(arrayTexts) > 1:
          doc.pop(idx)
          for text in arrayTexts:
            if len(arrayTexts) > 500:
              doc.append(text)
            else:
              doc.append(text)
    return doc

def pre_process(text):
    tokens = nltk.word_tokenize(text)
    tokens=[ t for t in tokens if False == t.isdigit()]
    tokens=[ t for t in tokens if False == containsNumeric(t)]
    tokens=[ t for t in tokens if True == t.isalpha()]
    text_after_process=" ".join(tokens)
    return(text_after_process)

def containsNumeric(text):
    if text is not None:
        return any(str.isdigit(c) for c in text)
    else:
        return True

In [ ]:
#Testing spacy and neuralcoref
#nlp = spacy.load('en_core_web_lg')
#neuralcoref.add_to_pipe(nlp)
#doc1 = nlp('My sister has a dog. She loves him.')
#print(doc1._.coref_clusters)
#print(doc1._.coref_clusters[1].mentions)
#print(doc1._.coref_clusters[1].mentions[-1])
#print(doc1._.coref_clusters[1].mentions[-1]._.coref_cluster.main)
#print(doc1._.coref_resolved)

#doc2 = nlp('Angela lives in Boston. She is quite happy in that city.')
#for ent in doc2.ents:
#    print(ent._.coref_cluster)

[My sister: [My sister, She], a dog: [a dog, him]]
[a dog, him]
him
a dog
My sister has a dog. My sister loves a dog.
Angela: [Angela, She]
Boston: [Boston, that city]


In [ ]:
spacy1 = spacy.load('en_core_web_lg')
nlp = spacy.load('en_core_web_lg')
neuralcoref.add_to_pipe(nlp)

tree = ET.parse('/content/drive/My Drive/PLP Sem 4/Restaurants_Train.xml')
root = tree.getroot()

labeled_reviews = []
for sentence in root.findall("sentence"):
    entry = {}
    aterms = []
    aspects = []
    if sentence.find("aspectTerms"):
        for aterm in sentence.find("aspectTerms").findall("aspectTerm"):
            aterms.append(aterm.get("term"))
    if sentence.find("aspectCategories"):
        for aspect in sentence.find("aspectCategories").findall("aspectCategory"):
            aspects.append(aspect.get("category"))
    entry["text"], entry["terms"], entry["aspects"]= sentence[0].text, aterms, aspects
    labeled_reviews.append(entry)
labeled_df = pd.DataFrame(labeled_reviews)
print("there are",len(labeled_reviews),"reviews in this training set")

labeled_df.to_pickle("annotated_reviews_df.pkl")
labeled_df.head()

there are 3044 reviews in this training set


,text,terms,aspects
0,But the staff was so horrible to us.,[staff],[service]
1,"To be completely fair, the only redeeming fact...",[food],"[food, anecdotes/miscellaneous]"
2,"The food is uniformly exceptional, with a very...","[food, kitchen, menu]",[food]
3,Where Gabriela personaly greets you and recomm...,[],[service]
4,"For those that go once and don't enjoy it, all...",[],[anecdotes/miscellaneous]


In [ ]:
annotated_reviews_df = pd.read_pickle("annotated_reviews_df.pkl")
annotated_reviews_df.head()

,text,terms,aspects
0,But the staff was so horrible to us.,[staff],[service]
1,"To be completely fair, the only redeeming fact...",[food],"[food, anecdotes/miscellaneous]"
2,"The food is uniformly exceptional, with a very...","[food, kitchen, menu]",[food]
3,Where Gabriela personaly greets you and recomm...,[],[service]
4,"For those that go once and don't enjoy it, all...",[],[anecdotes/miscellaneous]


In [ ]:
annotated_reviews_df["text_pro"] = annotated_reviews_df.text.map(lambda x: replace_pronouns(x))

annotated_reviews_df.to_pickle("annotated_reviews_df2.pkl")
annotated_reviews_df = pd.read_pickle("annotated_reviews_df2.pkl")

In [ ]:
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(annotated_reviews_df.aspects)
X = annotated_reviews_df.text_pro

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=0)

filename = 'mlb.pkl'
pickle.dump(mlb, open(filename, 'wb'))

In [ ]:
# Train naive bayes
text_clf = Pipeline([('vect', CountVectorizer(stop_words = "english",ngram_range=(1, 1))),
                     ('tfidf', TfidfTransformer(use_idf=False)),
                     ('clf', LabelPowerset(MultinomialNB(alpha=1e-1))),])
text_clf = text_clf.fit(X, y)

# save the model to disk
filename = 'naive_model1.pkl'
pickle.dump(text_clf, open(filename, 'wb'))


In [ ]:
data = pd.read_csv('/content/drive/My Drive/PLP Sem 4/reviews_with_food_subtypes.csv', encoding='unicode_escape')
data['aspect'] = ""
print(data.head())
print("Length of data is "+str(len(data)))

   Unnamed: 0  ... aspect
0           0  ...       
1           1  ...       
2           2  ...       
3           3  ...       
4           4  ...       

[5 rows x 13 columns]
Length of data is 202533


In [ ]:
print("Length of data is "+str(len(data)))
numReviewToDivide = 20000
parts = int(len(data)/numReviewToDivide)
startDataIdx = 0
endDataIdx = numReviewToDivide
data2 = data.iloc[startDataIdx:endDataIdx]
data2.head()

Length of data is 202533
10


,Unnamed: 0,Review,address,author,author_loc,date,origin,rating,restaurant,type,region,food_subtype,aspect
0,0,Visited on a day without Unagi *0*Location: 4/...,"7500A Beach Road The Plaza #B1-310, The Plaza,...",ML-SG-001,"Singapore, Singapore",8/10/2020,TripAdvisor,2,Uni Gallery by OosterBay,"Japanese,Sushi,Asian",CENTRAL,unagi#soup#fish#seafood#salad#egg#oyster#sushi...,
1,1,"Since my visit to Sushi Kimura, I had this str...","7500A Beach Road The Plaza #B1-310, The Plaza,...",morsels-of-delice,"Singapore, Singapore",10/30/2019,TripAdvisor,5,Uni Gallery by OosterBay,"Japanese,Sushi,Asian",CENTRAL,unagi#soup#fish#seafood#salad#egg#oyster#sushi...,
2,2,We enjoyed our meal very much. The uni platter...,"7500A Beach Road The Plaza #B1-310, The Plaza,...",baolover,"Singapore, Singapore",7/26/2020,TripAdvisor,5,Uni Gallery by OosterBay,"Japanese,Sushi,Asian",CENTRAL,unagi#soup#fish#seafood#salad#egg#oyster#sushi...,
3,3,"Came here during Stage 2 CB period , Oosterbay...","7500A Beach Road The Plaza #B1-310, The Plaza,...",Tan T,"Singapore, Singapore",7/23/2020,TripAdvisor,5,Uni Gallery by OosterBay,"Japanese,Sushi,Asian",CENTRAL,unagi#soup#fish#seafood#salad#egg#oyster#sushi...,
4,4,A highly recommended good Japanese restaurant ...,"7500A Beach Road The Plaza #B1-310, The Plaza,...",WSJJ,"Singapore, Singapore",7/3/2020,TripAdvisor,5,Uni Gallery by OosterBay,"Japanese,Sushi,Asian",CENTRAL,unagi#soup#fish#seafood#salad#egg#oyster#sushi...,


In [ ]:
column_names = ["Review", "address", "author", "author_loc", "date", "origin", "rating", "restaurant", "type", "region", "food_subtype", "aspect"]
dfWithAspects = pd.DataFrame(columns = column_names)

numReviewToDivide = 20000
#parts = int(len(data)/numReviewToDivide)
parts = 7
startDataIdx = 60000
endDataIdx = 80000

time_start = time.time()
for j in range(0, parts):
  if j < (parts-1):
    data2 = data.iloc[startDataIdx:endDataIdx]
  elif j == (parts-1):
    data2 = data.iloc[startDataIdx:-1]

  for index, row in data2.iterrows():
    print("Part "+str(j+4)+", index: "+str(index))
    res = row['Review']
    sentences = split_sentence(res)
    if len(sentences) > 1:
      for sentence in sentences:
        results = mlb.inverse_transform(loaded_model.predict([sentence]))
        listResults = results[0]
        if len(listResults) > 1:
          for result in listResults:
            if result != 'anecdotes/miscellaneous':
              dfWithAspects.loc[-1] = [pre_process(sentence), row["address"], row["author"], row["author_loc"], row["date"], row["origin"], row["rating"], row["restaurant"], row["type"], row["region"], row["food_subtype"], result]
              dfWithAspects.index = dfWithAspects.index + 1
              dfWithAspects = dfWithAspects.sort_index()
        elif len(listResults) == 1:
          if listResults[0] != 'anecdotes/miscellaneous':
            dfWithAspects.loc[-1] = [pre_process(sentence), row["address"], row["author"], row["author_loc"], row["date"], row["origin"], row["rating"], row["restaurant"], row["type"], row["region"], row["food_subtype"], listResults[0] ]
            dfWithAspects.index = dfWithAspects.index + 1
            dfWithAspects = dfWithAspects.sort_index()
    else:
        results = mlb.inverse_transform(loaded_model.predict([sentences[0]]))
        listResults = results[0]
        if len(listResults) > 1:
          for result in listResults:
            if result != 'anecdotes/miscellaneous':
              dfWithAspects.loc[-1] = [pre_process(sentences[0]), row["address"], row["author"], row["author_loc"], row["date"], row["origin"], row["rating"], row["restaurant"], row["type"], row["region"], row["food_subtype"], result]
              dfWithAspects.index = dfWithAspects.index + 1
              dfWithAspects = dfWithAspects.sort_index()
        elif len(listResults) == 1:
          if listResults[0] != 'anecdotes/miscellaneous':
            dfWithAspects.loc[-1] = [pre_process(sentences[0]), row["address"], row["author"], row["author_loc"], row["date"], row["origin"], row["rating"], row["restaurant"], row["type"], row["region"], row["food_subtype"], listResults[0] ]
            dfWithAspects.index = dfWithAspects.index + 1
            dfWithAspects = dfWithAspects.sort_index()
  dfWithAspects.to_csv('/content/drive/My Drive/PLP Sem 4/reviews_with_aspects_part_'+str(j+4)+'.csv')
  print("Saved part"+str(j+4)+" to CSV")
  startDataIdx += 20000
  endDataIdx += 20000
  data2 = data2.iloc[0:0]
  print(len(data2))
  dfWithAspects = dfWithAspects.iloc[0:0]
  print(len(dfWithAspects))

print('Processing done! Time elapsed: {} seconds'.format(time.time()-time_start))

Streaming output truncated to the last 5000 lines.
Part 10, index: 197537
Part 10, index: 197538
Part 10, index: 197539
Part 10, index: 197540
Part 10, index: 197541
Part 10, index: 197542
Part 10, index: 197543
Part 10, index: 197544
Part 10, index: 197545
Part 10, index: 197546
Part 10, index: 197547
Part 10, index: 197548
Part 10, index: 197549
Part 10, index: 197550
Part 10, index: 197551
Part 10, index: 197552
Part 10, index: 197553
Part 10, index: 197554
Part 10, index: 197555
Part 10, index: 197556
Part 10, index: 197557
Part 10, index: 197558
Part 10, index: 197559
Part 10, index: 197560
Part 10, index: 197561
Part 10, index: 197562
Part 10, index: 197563
Part 10, index: 197564
Part 10, index: 197565
Part 10, index: 197566
Part 10, index: 197567
Part 10, index: 197568
Part 10, index: 197569
Part 10, index: 197570
Part 10, index: 197571
Part 10, index: 197572
Part 10, index: 197573
Part 10, index: 197574
Part 10, index: 197575
Part 10, index: 197576
Part 10, index: 197577
Part 1

In [ ]:
dfWithAspects.to_csv('/content/drive/My Drive/PLP Sem 4/reviews_with_aspects.csv')